## Extracción de datos dinámica del sitio de TLC con reporte mensual.

De este sitio no hay vinculos nuevos cada cierto tiempo, sino que se usa el mismo vinculo para un nuevo archivo con más datos (suponiendo que el nombre del archivo siempre es el mismo).

Se puede descargar siempre del mismo link y con las transformaciones mantener los datos del mes nuevo exclusivamente para subirlos al Blob Storage como archivo nuevo.

In [0]:
%pip install fsspec

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
#Importar librerías.
import pandas as pd
import datetime
import pyspark

In [0]:
#Activar transferencia de datos columnares Arrow.
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [0]:
#URL del archivo que interesa.
url = "https://www1.nyc.gov/assets/tlc/downloads/csv/data_reports_monthly.csv"

### Acceso a Blob Storage

In [0]:
#Datos de acceso.
storage_account_name = "pfcrudo"
storage_account_key = "ASY9nV2+pddQN2Y8AUZef5m4JtdidWOa5RW+fNIkj2HE60F5s95ueJraFC2/Jv1KulbyL5hwcYk6+AStCn7cjw=="
container = "monthlyreports"

In [0]:
#Variable de configuración Spark.
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

In [0]:
#Lista de archivos ya en el almacenamiento.
file_list = dbutils.fs.ls(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/")

In [0]:
file_list

Out[7]: [FileInfo(path='wasbs://monthlyreports@pfcrudo.blob.core.windows.net/monthlyreports_2023_01.csv', name='monthlyreports_2023_01.csv', size=71781, modificationTime=1682120062000)]

Acceso al archivo más reciente de Azure Blob Storage:

In [0]:
#El índice de las listas de año y de mes deben coincidir con los de la file_list.
date_list = []

for file in file_list:
    year = file.name[15]+file.name[16]+file.name[17]+file.name[18]
    month = file.name[20]+file.name[21]
    date = year+month
    date_list.append(int(year+month))

#Año-mes mayor de la lista. Para obtener índice del elemento de file_list.
max = 0
i = 0

for date in date_list:
    if date > max:
        max = date
        index = i
    i += 1

max_date = max
date_index = index

In [0]:
#Dirección del archivo de Azure Blob Storage más reciente.
storage_url = file_list[date_index].path

In [0]:
#Leer y descargar archivo del almacenamiento.
storage_df = (spark.read.option("delimiter", ",").option("header", "true").csv(storage_url))

storage_df = storage_df.toPandas()

### Comparación de archivos

In [0]:
#Convertir de pandas dataframe a spark dataframe.
comparing_df = pd.read_csv(url)

In [0]:
storage_df

,Month/Year,License Class,Trips Per Day,Farebox Per Day,Unique Drivers,Unique Vehicles,Vehicles Per Day,Avg Days Vehicles on Road,Avg Hours Per Day Per Vehicle,Avg Days Drivers on Road,Avg Hours Per Day Per Driver,Avg Minutes Per Trip,Percent of Trips Paid with Credit Card,Trips Per Day Shared
0,2023-01,Green,"2,199","43,733",960,911,488,16.6,3.5,16.1,3.5,13.8,66%,-
1,2023-01,FHV - High Volume,"596,095",-,"77,641","74,553","50,996",21.2,6.4,20.7,6.3,18,-,"9,998"
2,2023-01,FHV - Black Car,"15,978",-,"7,197","6,809","2,877",13.1,4.3,12.8,4.2,25,-,-
3,2023-01,FHV - Livery,"18,853",-,"4,778","4,614","2,791",18.8,4.9,18.5,4.8,18,-,-
4,2023-01,FHV - Lux Limo,"1,060",-,"1,501","1,468",532,11.3,2.0,11.3,1.9,63,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,2010-05,Yellow,"499,374","5,561,218","30,950","13,309","12,729",29.6,14.8,22.7,8.3,-,35%,-
655,2010-04,Yellow,"504,798","5,502,545","30,827","13,326","12,856",28.9,14.8,22.3,8.3,-,35%,-
656,2010-03,Yellow,"415,567","3,011,750","28,709","12,183","10,382",26.4,15.1,20.4,8.3,11.5,35%,-
657,2010-02,Yellow,"397,969","3,392,416","28,255","12,171","10,045",23.1,14.6,17.6,8.3,11.1,35%,-


In [0]:
tuple_a = comparing_df.apply(tuple, 1)
tuple_b = storage_df.apply(tuple, 1)

#Cuáles tuplas se encuentran en el nuevo dataframe (True = Sí están, False = No están).
~tuple_a.isin(tuple_b)

Out[18]: 0      True
1      True
2      True
3      True
4      True
       ... 
654    True
655    True
656    True
657    True
658    True
Length: 659, dtype: bool

In [0]:
#Obtener las filas que discrepan.
result = comparing_df[tuple_a.isin(tuple_b)]

In [0]:
#Número de filas del resultado.
rows = len(result.index)

In [0]:
comparing_df

,Month/Year,License Class,Trips Per Day,Farebox Per Day,Unique Drivers,Unique Vehicles,Vehicles Per Day,Avg Days Vehicles on Road,Avg Hours Per Day Per Vehicle,Avg Days Drivers on Road,Avg Hours Per Day Per Driver,Avg Minutes Per Trip,Percent of Trips Paid with Credit Card,Trips Per Day Shared
0,2023-01,Green,"2,199","43,733",960,911,488,16.6,3.5,16.1,3.5,13.8,66%,-
1,2023-01,FHV - High Volume,"596,095",-,"77,641","74,553","50,996",21.2,6.4,20.7,6.3,18,-,"9,998"
2,2023-01,FHV - Black Car,"15,978",-,"7,197","6,809","2,877",13.1,4.3,12.8,4.2,25,-,-
3,2023-01,FHV - Livery,"18,853",-,"4,778","4,614","2,791",18.8,4.9,18.5,4.8,18,-,-
4,2023-01,FHV - Lux Limo,"1,060",-,"1,501","1,468",532,11.3,2.0,11.3,1.9,63,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,2010-05,Yellow,"499,374","5,561,218","30,950","13,309","12,729",29.6,14.8,22.7,8.3,-,35%,-
655,2010-04,Yellow,"504,798","5,502,545","30,827","13,326","12,856",28.9,14.8,22.3,8.3,-,35%,-
656,2010-03,Yellow,"415,567","3,011,750","28,709","12,183","10,382",26.4,15.1,20.4,8.3,11.5,35%,-
657,2010-02,Yellow,"397,969","3,392,416","28,255","12,171","10,045",23.1,14.6,17.6,8.3,11.1,35%,-


In [0]:
#Obtener la fecha de los nuevos datos, usando la moda de esa columna (Month/Year).
new_date = list(comparing_df["Month/Year"].mode())[0]

#Nombrar el archivo que se guardará.
name_year = new_date[0]+new_date[1]+new_date[2]+new_date[3]
name_month = new_date[5]+new_date[6]
file_name = (f"monthlyreports_{name_year}_{name_month}")

### Exportación a Blob Storage

In [0]:
#Si el resultado tiene filas, exportarlo a almacenamiento.
if rows > 0:
    result_df = spark.createDataFrame(result)
    result_df.write.format("csv").save(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/{file_name}")
else:
    print("No hay diferencias en los datos.")